In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import shutil
ROOT_DIR = '/content/gdrive/yolov4'
#ROOT_DIR = "/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project"
CONFIG_PATH = os.path.join(ROOT_DIR,"datasets/config")
TEMPLATE_PATH = os.path.join(CONFIG_PATH,'template')
BACKUP_PATH = os.path.join(CONFIG_PATH,'backup')
TRAIN_PATH = os.path.join(CONFIG_PATH,'train.txt')
VALID_PATH = os.path.join(CONFIG_PATH,'val.txt')


DATA_FILE = "obj.data"
NAMES_FILE = "obj.names"

DATA_PATH = os.path.join(CONFIG_PATH,DATA_FILE)
NAMES_PATH = os.path.join(CONFIG_PATH,NAMES_FILE)

CLS = ['face']
CLS_NUM = len(CLS)

In [ ]:
%cd '{ROOT_DIR}'
#!git clone https://github.com/AlexeyAB/darknet.git

/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project


In [ ]:
%cd darknet
!make clean

/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/darknet
rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/representation_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/rnn.o ./obj/rnn_vid.o ./obj/crnn_layer.o ./obj/demo.o ./obj/tag.o ./obj/cifar.o ./obj/g

In [ ]:
!cp -f '{TEMPLATE_PATH}/{DATA_FILE}' '{DATA_PATH}'

!sed -i "s|classes_num|{CLS_NUM}|g" '{DATA_PATH}'
!sed -i "s|train_path|{TRAIN_PATH}|g" '{DATA_PATH}'
!sed -i "s|valid_path|{VALID_PATH}|g" '{DATA_PATH}'
!sed -i "s|names_path|{NAMES_PATH}|g" '{DATA_PATH}'
!sed -i "s|backup_path|{BACKUP_PATH}|g" '{DATA_PATH}'

f = open('obj-names.txt','w')
for i,c in enumerate(CLS):
  f.write(c)
  if i != CLS_NUM-1:
    f.write('\n')

f.close()

!mv -f '{f.name}' '{NAMES_PATH}'

if not shutil.os.path.exists(BACKUP_PATH):
  shutil.os.mkdir(BACKUP_PATH)

In [ ]:
%cd darknet

/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/darknet


In [ ]:
!sed -i "s/GPU=0/GPU=1/g" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/g" Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" Makefile

In [ ]:
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv

In [ ]:
#!%cp "{ROOT_DIR}/datasets/labels/*.txt" "{ROOT_DIR}/datasets/Images"

In [ ]:

#!ln -s '/content/gdrive/MyDrive/Colab Notebooks' '/content/gdrive/MyDrive/ColabNotebooks'
!./darknet detector calc_anchors '$DATA_PATH' -num_of_clusters 9 -width 416 -height 416 -showpause

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 416, height = 416 
 read labels from 12876 images 
 loaded 	 image: 12876 	 box: 156994
 all loaded. 

 calculating k-means++ ...

 iterations = 217 


counters_per_class = 156994

 avg IoU = 70.67 % 

Saving anchors to the file: anchors.txt 
anchors =   3,  5,   5,  9,   8, 15,  13, 24,  22, 38,  37, 59,  61,101, 109,158, 183,242
^C


In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
WEIGHTS_PATH = os.path.join(BACKUP_PATH,'yolov4-custom_last.weights')
RETRAIN = False
if (not os.path.exists(WEIGHTS_PATH)) and RETRAIN:
  WEIGHTS_PATH = 'yolov4.conv.137'
print(WEIGHTS_PATH)
CFG_PATH = os.path.join(CONFIG_PATH,'yolov4-custom.cfg')

/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/datasets/config/backup/yolov4-custom_last.weights


In [ ]:
print(CFG_PATH)
print(WEIGHTS_PATH)
print(TRAIN_PATH)
print(CONFIG_PATH)

/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/datasets/config/yolov4-custom.cfg
/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/datasets/config/backup/yolov4-custom_last.weights
/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/datasets/config/train.txt
/content/gdrive/MyDrive/Colab Notebooks/HMI/final_project/datasets/config


In [ ]:
!./darknet detector train '$DATA_PATH' '$CFG_PATH' '$WEIGHTS_PATH' -map -dont_show

串流輸出內容已截斷至最後 5000 行。
 total_bbox = 440131, rewritten_bbox = 1.790149 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.644601), count: 1, class_loss = 0.197873, iou_loss = 2.708121, total_loss = 2.905993 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.593576), count: 4, class_loss = 0.888520, iou_loss = 5.044708, total_loss = 5.933228 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.698706), count: 10, class_loss = 1.014137, iou_loss = 0.908590, total_loss = 1.922726 
 total_bbox = 440146, rewritten_bbox = 1.790088 % 

 (next mAP calculation at 1704 iterations) ]2;1103/6000: loss=4.7 hours left=24.8
 1103: 4.711815, 5.809497 avg loss, 0.001000 rate, 12.285244 seconds, 70592 images, 24.769635 hours left
Loaded: 0.000078 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.624670), count: 4, class_loss = 1.083502, iou_loss = 25.927801, total_l

In [ ]:
sample_path = ROOT_DIR+'/datasets/config/74098_01.jpg'
if os.path.exists(sample_path):
  print("exists")
!./darknet detector test '$DATA_PATH' '$CFG_PATH' '$WEIGHTS_PATH' '{sample_path}' -thresh 0.3 -dont_show

exists
 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8